## 目录

本文主要内容如下：
1. 从使用的角度来接触模型
2. 本地运行的方式来认识模型
3. 以文本生成过程来理解模型
4. 以内部窥探的方式来解剖模型

## 1. 模型前台使用

#### 可视化UI的方式使用模型

[点此进入](http://192.168.31.200:8501/)

#### 使用openai库来访问模型

In [4]:
from openai import OpenAI

def predict(prompt):
    client = OpenAI(api_key="0",base_url="http://192.168.31.200:8000/v1")
    messages = [{"role": "user", "content": prompt}]
    result = client.chat.completions.create(messages=messages, model="Qwen2-0.5B-Instruct")
    return result.choices[0].message.content

predict("详细介绍下你自己。")

'我是阿里云研发的超大规模语言模型，我叫通义千问。'





上面的`client.chat.completions.create`方法调用本质上是发送了类似下面一个请求。

In [5]:
!curl -s http://192.168.31.200:8000/v1/chat/completions \
    -H "Content-Type: application/json" \
    -d '{ \
        "model": "Qwen2-0.5B-Instruct",\
        "messages": [{"role": "user", "content": "详细介绍下你自己。"}],\
        "max_tokens": 512,\
        "temperature": 0.7  \
    }' | jq

{
  "id": "chat-373150f974fd4367b53632817ed44cbc",
  "object": "chat.completion",
  "created": 1730414253,
  "model": "Qwen2-0.5B-Instruct",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "我是阿里云开发的一款超大规模语言模型，我叫通义千问。",
        "tool_calls": []
      },
      "logprobs": null,
      "finish_reason": "stop",
      "stop_reason": null
    }
  ],
  "usage": {
    "prompt_tokens": 23,
    "total_tokens": 41,
    "completion_tokens": 18
  },
  "prompt_logprobs": null
}


> 上面是OpenAI接口的完整response，在聊天窗口中显示的推理结果只是取了`response['choices'][0]['message']['content']`。

## 2. 模型后台运行

模型的后台运行我们可以简单分为如下四步：
1. 标准化提示语
2. 序列化提示语
3. 模型预测
4. 反序列化为文本

#### 2.1 加载模型
加载模型最基本的方式是使用 `transformers` 库，它是由 Hugging Face 开发的一个开源库，它提供了很方便的方式来访问和使用各种自然语言处理（NLP）模型，也提供了便捷的API来微调和训练开源模型。

> 目前业界开源的语言模型基本都会发布到HuggingFace，也都支持transformers库来加载。huggingface上不仅提供模型、开发库，还提供数据集甚至算力。

In [1]:
import os
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

# 定义模型本地路径和要加载到的目标设备
device = "cuda:4" 
model_path = "/data2/anti_fraud/models/modelscope/hub/Qwen/Qwen2-0___5B-Instruct"

In [9]:
!ls -l /data2/anti_fraud/models/modelscope/hub/Qwen/Qwen2-0___5B-Instruct

total 976188
-rw-rw-r-- 1 xiaoguanghua xiaoguanghua       659 Aug  1 18:26 config.json
-rw-rw-r-- 1 xiaoguanghua xiaoguanghua        48 Aug  1 18:26 configuration.json
-rw-rw-r-- 1 xiaoguanghua xiaoguanghua       242 Aug  1 18:26 generation_config.json
-rw-rw-r-- 1 xiaoguanghua xiaoguanghua     11344 Aug  1 18:26 LICENSE
-rw-rw-r-- 1 xiaoguanghua xiaoguanghua   1671839 Aug  1 18:26 merges.txt
-rw-rw-r-- 1 xiaoguanghua xiaoguanghua 988097824 Aug  1 18:31 model.safetensors
-rw-rw-r-- 1 xiaoguanghua xiaoguanghua      3551 Aug  1 18:31 README.md
-rw-rw-r-- 1 xiaoguanghua xiaoguanghua      1287 Aug  1 18:31 tokenizer_config.json
-rw-rw-r-- 1 xiaoguanghua xiaoguanghua   7028015 Aug  1 18:31 tokenizer.json
-rw-rw-r-- 1 xiaoguanghua xiaoguanghua   2776833 Aug  1 18:31 vocab.json


In [2]:
model = AutoModelForCausalLM.from_pretrained(model_path, torch_dtype=torch.bfloat16).to(device)
tokenizer = AutoTokenizer.from_pretrained(model_path, use_fast=False, trust_remote_code=True)

- AutoModelForCausalLM是用于加载因果语言模型，这类模型在生成新词时只依赖于之前生成的词，而不考虑未来的词。

- AutoTokenizer是用于文本分词的组件，它能根据模型路径自动加载词表。

#### 2.2 标准化提示语

In [3]:
prompt = "详细介绍下你自己。"
messages = [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": prompt}
]
# 将消息应用到标准模板，使结构一致
input_text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True,
)
print(f"{input_text}") 


<|im_start|>system
You are a helpful assistant.<|im_end|>
<|im_start|>user
详细介绍下你自己。<|im_end|>
<|im_start|>assistant



**疑问**：为什么不使用原先的json分隔符，而要单独定义一套标记作为分隔符呢？

In [23]:
tokenizer.tokenize("congratulation")

['con', 'gr', 'at', 'ulation']

#### 2.3 提示语序列化
序列化的本质就是把人类理解的文本转换为模型可以计算的数字。
![tokenize.png](./img/认识模型/tokenize.png)


In [4]:
model_inputs = tokenizer([input_text], return_tensors="pt").to(device)
print(f"{model_inputs}")

{'input_ids': tensor([[151644,   8948,    198,   2610,    525,    264,  10950,  17847,     13,
         151645,    198, 151644,    872,    198, 113511,  16872, 107828,   1773,
         151645,    198, 151644,  77091,    198]], device='cuda:4'), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]],
       device='cuda:4')}


- return_tensors='pt'表示以pytorch张量的形式返回，并通过'.to(device)'来将数据移动到指定的GPU设备上，便于后续计算。

- attention_mask主要用于批量预测场景，用于计算注意力时屏蔽填充token。

In [ ]:
len(tokenizer.encoder)

#### 2.4 模型预测生成

`model.generate`函数用于对一个序列进行推理，并输出预测的新序列。

In [5]:
response_ids = model.generate(
    model_inputs.input_ids,
    max_new_tokens=512,
)
print(f"response_ids: {response_ids}")

The attention mask is not set and cannot be inferred from input because pad token is same as eos token.As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


response_ids: tensor([[151644,   8948,    198,   2610,    525,    264,  10950,  17847,     13,
         151645,    198, 151644,    872,    198, 113511,  16872, 107828,   1773,
         151645,    198, 151644,  77091,    198, 104198, 101919, 102661,  99718,
         104197, 100176, 102064, 104949,   3837,  35946,  99882,  31935,  64559,
          99320,  56007,   1773, 151645]], device='cuda:4')


#### 2.5 反序列化为文本

In [6]:
response_ids = [
    output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, response_ids)
]
print(f"response_ids: {response_ids}")
response_text = tokenizer.batch_decode(response_ids, skip_special_tokens=True)[0]
print(f"{response_text}")

response_ids: [tensor([104198, 101919, 102661,  99718, 104197, 100176, 102064, 104949,   3837,
         35946,  99882,  31935,  64559,  99320,  56007,   1773, 151645],
       device='cuda:4')]
我是来自阿里云的大规模语言模型，我叫通义千问。


> 模型输出的序列response_ids，是同时包含输入和输出的完整序列，需要后期处理来截掉输入序列，只保留输出序列。

> 模型的输出结果中会包含特殊token（例如：<|im_end|>），需要通过skip_special_tokens=False过滤，只保留真正有效的文本。


In [ ]:
tokenizer.decoder

上面是模型生成文本的一个过程。

这里有一个疑问：模型是一次性生成整个序列吗？



## 3. model.generate的运行探究

实际上model.generate并不是pytorch的标准API，而是为了方便调用而高度封装过的，`model.forward`才是pytorch模型标准调用方法。
> PyTorch 是Facebook开发一个深度学习框架，它相比于Tensorflow来说更加容易学习和上手，其发展速度已经超过了Tensorflow。

#### 3.1 手动模拟
我们尝试还原一下`model.generate`方法的运行过程。

In [7]:
# 定义最终的序列，初始值为输入序列
generate_ids = model_inputs.input_ids

In [ ]:
定义一个函数`generate_next_token`,作用是预测一个token。

In [8]:
def generate_next_token(model, generate_ids, temperature=0.7, debug=False):
    print("input_ids:", generate_ids) if debug else None
    logits = model.forward(generate_ids).logits
    print("logits: ", logits) if debug else None
    
    if temperature > 0:
        probs = torch.softmax(logits[:, -1] / temperature, dim = -1)
        print("probs: ", len(probs[0]), probs) if debug else None
        next_token = torch.multinomial(probs[-1], num_samples=1)  # 按照概率采样得到下一个token
    else:
        next_token = torch.argmax(logits[:, -1], dim=-1)
    
    print("next_id: ", next_token, ", token: ", tokenizer.decode(next_token)) if debug else None
    return next_token.reshape(-1, 1)

- `model.forward`是为了执行矩阵运算，得到未归一化的概率。
- `torch.softmax`是为了得到归一化的概率（所有可能值的概率之和为1），`logits[:, -1]`表示取三维张量的第二维最后一列，即最后一个token的张量，用于预测下一个token。
- `temperature`在这里起到的作用是：温度越高（大于1），概率分布越平滑，温度越低（小于1），概率分布越尖锐。
- `torch.multinomial`：按照概率大小进行随机加权采样，num_samples=1表示只采样一个值作为结果返回。

In [11]:
next_token = generate_next_token(model, generate_ids, debug=True)
generate_ids = torch.cat((generate_ids, next_token), dim=1)
tokenizer.batch_decode(generate_ids[:, len(model_inputs.input_ids[0]):], skip_special_tokens=True)[0] 

input_ids: tensor([[151644,   8948,    198,   2610,    525,    264,  10950,  17847,     13,
         151645,    198, 151644,    872,    198, 113511,  16872, 107828,   1773,
         151645,    198, 151644,  77091,    198, 104198, 101919]],
       device='cuda:4')
logits:  tensor([[[ 2.4219,  2.6250,  2.9531,  ..., -3.3281, -3.3281, -3.3281],
         [ 0.5938,  1.8359,  3.6250,  ..., -4.5000, -4.5000, -4.5000],
         [ 8.0625,  4.5938, 10.8125,  ..., -3.4531, -3.4531, -3.4531],
         ...,
         [ 6.0312, 10.1250,  5.4062,  ..., -3.4062, -3.3906, -3.3906],
         [ 2.5781,  6.5000,  3.3906,  ..., -5.1562, -5.1562, -5.1562],
         [ 2.0781,  6.0938,  3.8438,  ..., -5.9062, -5.8750, -5.8750]]],
       device='cuda:4', grad_fn=<ToCopyBackward0>)
probs:  151936 tensor([[5.4616e-10, 1.6931e-07, 6.8037e-09,  ..., 6.0764e-15, 6.3538e-15,
         6.3538e-15]], device='cuda:4', grad_fn=<SoftmaxBackward0>)
next_id:  tensor([102661], device='cuda:4') , token:  阿里


'我是来自阿里'

>`torch.cat`是用于将下一个token追加到序列的末尾，作为新的上下文重新输入给模型。


> 通过generate_next_token产生下一个token后，需要将这个token追加到输入序列的结尾，继续预测下下一个token。预测到下下一个token后，继续追加到输入序列的结尾……如此循环，来逐步生成一个完整的序列。

#### 3.2 自动化
将上面的手动一次一次运行的过程，写成一个循环，一次性输出整个序列。

In [13]:
tokenizer

Qwen2Tokenizer(name_or_path='/data2/anti_fraud/models/modelscope/hub/Qwen/Qwen2-0___5B-Instruct', vocab_size=151643, model_max_length=32768, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'eos_token': '<|im_end|>', 'pad_token': '<|endoftext|>', 'additional_special_tokens': ['<|im_start|>', '<|im_end|>']}, clean_up_tokenization_spaces=False),  added_tokens_decoder={
	151643: AddedToken("<|endoftext|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	151644: AddedToken("<|im_start|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	151645: AddedToken("<|im_end|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}

In [15]:
import time
max_new_tokens = 128
end_token_id = 151645
generate_ids = model_inputs.input_ids

for _ in range(max_new_tokens):
    next_token = generate_next_token(model, generate_ids, debug=False)
    generate_ids = torch.cat((generate_ids, next_token), dim=1)
    if next_token.item() == end_token_id:
        break
    print(tokenizer.decode(next_token[0], skip_special_tokens=False), end='')
    time.sleep(0.1)

作为一个人工智能，我是一个计算机程序，没有情感、意识或身体，因此我无法有自己的思想、感情或身体。我被设计为帮助用户解答问题、提供信息和完成任务。我每天都会不断学习和更新，以便更好地服务用户。

In [ ]:
tokenizer.added_tokens_decoder

疑问：模型为什么要一个token一个token生成，而不是一遍矩阵运算生成多个词呢？

#### 3.3 自回归

像上面这样，每生成一个token，都追加到原上下文的末尾，作为新的上下文来预测下一个token这种生成方式，就叫`自回归解码`。

> 采用自回归解码的模型被称为因果语言模型，这类模型预测下一个token只依赖前面的token，也是上面`AutoModelForCausalLM`这个名字的由来。



<img src="./img/认识模型/auto_regression.png" width="700px" height="700px"/>


## 4. model内部是什么？

#### 4.1 主体结构

In [17]:
print(model)

Qwen2ForCausalLM(
  (model): Qwen2Model(
    (embed_tokens): Embedding(151936, 896)
    (layers): ModuleList(
      (0-23): 24 x Qwen2DecoderLayer(
        (self_attn): Qwen2SdpaAttention(
          (q_proj): Linear(in_features=896, out_features=896, bias=True)
          (k_proj): Linear(in_features=896, out_features=128, bias=True)
          (v_proj): Linear(in_features=896, out_features=128, bias=True)
          (o_proj): Linear(in_features=896, out_features=896, bias=False)
          (rotary_emb): Qwen2RotaryEmbedding()
        )
        (mlp): Qwen2MLP(
          (gate_proj): Linear(in_features=896, out_features=4864, bias=False)
          (up_proj): Linear(in_features=896, out_features=4864, bias=False)
          (down_proj): Linear(in_features=4864, out_features=896, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): Qwen2RMSNorm()
        (post_attention_layernorm): Qwen2RMSNorm()
      )
    )
    (norm): Qwen2RMSNorm()
  )
  (lm_head): Linear(in_featur

Attention+MLP参数量估算：
- W(q)参数量 = 896 × 896 + 896 = 803712
- W(k)参数量 = 896 × 128 + 128 = 115584
- W(v)参数量 = 896 × 128 + 128 = 115584
- W(o)参数量 = 896 × 896 = 802816
- W(gate)参数量 = 896 × 4864 = 4358144
- W(up)参数量 = 896 × 4864 = 4358144
- W(down)参数量 = 4864 × 896 = 4358144

DecoderLayer参数量估算
- W(DecoderLayer) = W(q) + W(k) + W(v) + W(o) + W(gate) + W(up) + W(down) = 14911928
- W(lm_head)参数量 = 896 × 151936 = 136134656

模型总参数量估算：
- W(model) = W(Decoderlayer) * 24 + W(lm_head) = 357886272 + 136134656 = 494020928

In [18]:
model_path_7b = "/data2/anti_fraud/models/modelscope/hub/Qwen/Qwen2-7B-Instruct"
model_7b = AutoModelForCausalLM.from_pretrained(model_path_7b, torch_dtype=torch.bfloat16).to(device)
model_7b

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Qwen2ForCausalLM(
  (model): Qwen2Model(
    (embed_tokens): Embedding(152064, 3584)
    (layers): ModuleList(
      (0-27): 28 x Qwen2DecoderLayer(
        (self_attn): Qwen2SdpaAttention(
          (q_proj): Linear(in_features=3584, out_features=3584, bias=True)
          (k_proj): Linear(in_features=3584, out_features=512, bias=True)
          (v_proj): Linear(in_features=3584, out_features=512, bias=True)
          (o_proj): Linear(in_features=3584, out_features=3584, bias=False)
          (rotary_emb): Qwen2RotaryEmbedding()
        )
        (mlp): Qwen2MLP(
          (gate_proj): Linear(in_features=3584, out_features=18944, bias=False)
          (up_proj): Linear(in_features=3584, out_features=18944, bias=False)
          (down_proj): Linear(in_features=18944, out_features=3584, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): Qwen2RMSNorm()
        (post_attention_layernorm): Qwen2RMSNorm()
      )
    )
    (norm): Qwen2RMSNorm()
  )
  (lm_head): Lin

> 每生成一个token，都要进行24层的解码器运算，每层解码器都要有14911928个参数进行运算，所以语言模型的推理功能对GPU运算的消耗是巨大的。

#### 4.2 组件架构
模型的整个结构基本如下图所示：

![framework.jpg](./img/认识模型/framework.jpg)

- `hidden_states`: 隐藏状态，除了输入和输出外，中间计算结果都称为隐藏状态，它会随着每层解码器的运算而不断更新，更新后再作为下一层解码器的输入，原始值为embedding后的输入向量，经过所有解码器运算完后作为最终输出。
- `RMSNorm`: 是一种归一化技术，它能将序列中的所有值都规范化到同一尺度，以稳定模型的训练过程。
- `residual`: 残差连接，通过在神经网络的某些层之间添加跨层连接，将输入信号直接传递到后续层，目的是为了缓解训练过程中的梯度消失问题，并学习数据之间的长期依赖关系，它假设当前的输出与之前或更以前的输入序列之间存在着依赖关系。
- `MLP`: 多层感知器，是一个前馈神经网络（Feed forward network)，通过维度放大来增加信息的丰富程度，再通过维度缩小来对提取的特征进行压缩以捕捉到不同层次的抽象特征，并通过激活函数引入非线性变换来提高模型的表达能力。
- `Attention`: 注意力机制，目的是为了获得每个单词在一个句子(上下文)中的含义，它的作法是计算`每个词与序列中其它词之间的相关度得分`（即注意力），它假设人类是`通过句子中某几个关键词来理解一句话中某个词的含义`。



两个例句对比：
1. 明天 早点 来。   --> 时间
2. 你吃早点了吗？ --> 早餐

自注意力中的Q、K、V理解：
- Q(Query)：表示的是查询，可以理解当前每个输入位置的信息本身，以前面的句子为例，要衡量`早点`对`来`的注意力时，`早点`就是Q； 
- K(Key)：表示输入序列中每个位置的索引，即`来`这个词所在的位置； 
- V(Value)：表示与每个位置（K）相关联的内容，它能用来表示实际要被关注并对后续处理有价值的信息； 
- K和Q的点积表示词元与词元之间两两注意力权重，它决定了哪些输入位置的Value信息对当前Query最重要，假如`来`对`早点`这个token最重要，那`早点`和`来`的点积就会比较大；
- AttentionWeight(注意力权重)再与V加权求和，就得到一个充分理解了每个单词上下文含义的新向量。

疑问：模型不应该是编码器——解码器架构吗？



<img src="./img/认识模型/transformer.png" width="450px" height="450px"/>

除了结构比仅解码器架构更复杂外，编码器-解码器架构的最大的局限性在于，解码器在解码时只能依赖于编码器输出的最后隐藏状态，该架构期望的是编码器输出的隐藏状态能完全表示输入序列的所有信息。

但实际上这个很难做到，当这个隐藏状态信息不足以表示整个输入序列时，而解码器又无法直接访问原始序列和编码器中更早期的隐藏状态，最终就会导致上下文丢失。

transformer分为三种架构：
- encoder-decoder
- encoder-only
- decoder-only

<img src="./img/认识模型/llm_history.png" width="900px" height="900px"/>

#### 4.3 模型参数

In [21]:
model

Qwen2ForCausalLM(
  (model): Qwen2Model(
    (embed_tokens): Embedding(151936, 896)
    (layers): ModuleList(
      (0-23): 24 x Qwen2DecoderLayer(
        (self_attn): Qwen2SdpaAttention(
          (q_proj): Linear(in_features=896, out_features=896, bias=True)
          (k_proj): Linear(in_features=896, out_features=128, bias=True)
          (v_proj): Linear(in_features=896, out_features=128, bias=True)
          (o_proj): Linear(in_features=896, out_features=896, bias=False)
          (rotary_emb): Qwen2RotaryEmbedding()
        )
        (mlp): Qwen2MLP(
          (gate_proj): Linear(in_features=896, out_features=4864, bias=False)
          (up_proj): Linear(in_features=896, out_features=4864, bias=False)
          (down_proj): Linear(in_features=4864, out_features=896, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): Qwen2RMSNorm()
        (post_attention_layernorm): Qwen2RMSNorm()
      )
    )
    (norm): Qwen2RMSNorm()
  )
  (lm_head): Linear(in_featur

In [ ]:
params = model.state_dict()
params

In [20]:
params['model.layers.0.self_attn.q_proj.weight']

tensor([[-0.0024, -0.0156,  0.0229,  ..., -0.0079, -0.0128,  0.0023],
        [ 0.0200,  0.0100, -0.0208,  ...,  0.0234,  0.0090, -0.0013],
        [-0.0151, -0.0035,  0.0219,  ..., -0.0092, -0.0283,  0.0043],
        ...,
        [-0.0415, -0.0522, -0.0233,  ...,  0.0427, -0.0374,  0.0063],
        [-0.0398, -0.0284, -0.0083,  ...,  0.0378, -0.0408, -0.0204],
        [ 0.0491, -0.0129,  0.0015,  ..., -0.0439,  0.0197,  0.0352]],
       device='cuda:4', dtype=torch.bfloat16)

**微调的目的**：对这些参数中的全部或部分进行调整，以便它能更适配目标场景的任务。

**小结**：本文以使用模型作为起点，由前台到后台，由外及里，逐层解开模型生成文本的整个过程。目的是让学习者快速理解生成式语言模型的基本工作原理，在这个过程中，我们详细讨论了文本生成的细节，并提供了可操作的代码块，以方便感兴趣的同学自己动手尝试。